In [1]:
!pip install pubchempy
!pip install rdkit==2024.3.5

  Preparing metadata (setup.py) ... done
  Created wheel for pubchempy: filename=PubChemPy-1.0.4-py3-none-any.whl size=13819 sha256=d789788f36858a9832666bb456a1c185b2635fe003d2307f30dab3c50628d945
  Stored in directory: /root/.cache/pip/wheels/90/7c/45/18a0671e3c3316966ef7ed9ad2b3f3300a7e41d3421a44e799
Successfully built pubchempy
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.1/33.1 MB 34.5 MB/s eta 0:00:00


In [2]:
import pandas as pd
import pubchempy as pcp
from rdkit import Chem

In [5]:
# Load the CSV file
csv_file_path = '/content/new_chemicals_100.csv'
try:
    chemicals_df = pd.read_csv(csv_file_path)
except FileNotFoundError:
    print(f"Error: The file '{csv_file_path}' was not found.")
    exit()
except pd.errors.EmptyDataError:
    print(f"Error: The file '{csv_file_path}' is empty.")
    exit()
except Exception as e:
    print(f"Error loading CSV file: {e}")
    exit()

# Automatically detect the first column name
column_name = chemicals_df.columns[0]

# Check if DataFrame is empty
if chemicals_df.empty:
    print("Error: The CSV file contains no data.")
    exit()

# Lists to store accepted and rejected chemicals
accepted_chemicals = []
rejected_chemicals = []

# Function to find similar compounds and generate their SMILES with dynamic threshold adjustment
def find_similar_smiles(chemical_name):
    try:
        # Search for the compound on PubChem
        original_compounds = pcp.get_compounds(chemical_name, 'name')

        if not original_compounds or len(original_compounds) == 0:
            print(f"No compound found for: {chemical_name}")
            return []

        # Get the CID of the original compound
        original_cid = original_compounds[0].cid

        # Start with a high similarity threshold and decrease if no valid compound is found
        threshold = 90
        min_threshold = 20
        step = 10
        smiles_list = []

        while threshold >= min_threshold and len(smiles_list) == 0:
            try:
                # Find similar compounds using the original compound's CID
                similar_compounds = pcp.get_similar_compounds(original_cid, threshold=threshold)[:5]  # Retrieve up to 5 similar compounds

                for similar_compound in similar_compounds:
                    # Get the canonical SMILES of the similar compound
                    smiles = similar_compound.canonical_smiles
                    # Validate using RDKit
                    mol = Chem.MolFromSmiles(smiles)
                    if mol:
                        smiles_list.append(smiles)

            except Exception as e:
                print(f"Error retrieving similar compounds at threshold {threshold}: {e}")

            # Decrease the threshold if no valid compounds found
            if len(smiles_list) == 0:
                print(f"No valid similar compounds found at threshold {threshold}. Decreasing threshold.")
                threshold -= step

        return smiles_list

    except Exception as e:
        print(f"Error processing {chemical_name}: {e}")
        return []

# Loop through each chemical name to generate SMILES
for index, row in chemicals_df.iterrows():
    print(f"Processing index {index}")
    chemical_name = row[column_name]

    try:
        # Use PubChemPy to get the compound and SMILES representation
        compound = pcp.get_compounds(chemical_name, 'name')
        smiles = None

        # Check if compound was found and has a canonical SMILES
        if compound and len(compound) > 0 and compound[0].canonical_smiles:
            smiles = compound[0].canonical_smiles

            # Validate the SMILES using RDKit
            mol = Chem.MolFromSmiles(smiles)
            if mol:
                accepted_chemicals.append({'Chemical_Name': chemical_name, 'SMILES': smiles})
            else:
                smiles = None

        # If no valid SMILES was found, search for similar chemicals
        if not smiles:
            similar_smiles = find_similar_smiles(chemical_name)
            if similar_smiles:
                for smiles in similar_smiles:
                    accepted_chemicals.append({'Original_Chemical_Name': chemical_name, 'Chemical_Name': f"{chemical_name} (similar)", 'SMILES': smiles})
            else:
                rejected_chemicals.append({'Chemical_Name': chemical_name})

    except Exception as e:
        print(f"Error processing {chemical_name}: {e}")
        rejected_chemicals.append({'Chemical_Name': chemical_name})

# Convert lists to DataFrames
accepted_df = pd.DataFrame(accepted_chemicals)
rejected_df = pd.DataFrame(rejected_chemicals)

# Save the accepted chemical names along with their SMILES representations
accepted_csv_file_path = 'accepted_chemicals.csv'
accepted_df.to_csv(accepted_csv_file_path, index=False)

# Save the rejected chemical names
rejected_csv_file_path = 'rejected_chemicals.csv'
rejected_df.to_csv(rejected_csv_file_path, index=False)

# Display a sample of the accepted and rejected DataFrames
print("Accepted Chemicals:")
print(accepted_df.head())
print("\nRejected Chemicals:")
print(rejected_df.head())

Processing index 0
No compound found for: Fluorotanil
Processing index 1
Processing index 2
Processing index 3
Processing index 4
Processing index 5
Processing index 6
Processing index 7
Processing index 8
Processing index 9
Processing index 10
Processing index 11
Processing index 12
Processing index 13
Processing index 14
Processing index 15
No compound found for: Denosumab
Processing index 16
No compound found for: Etanercept
Processing index 17
No compound found for: Adalimumab
Processing index 18
No compound found for: Infliximab
Processing index 19
Error processing Tocilizumab: 'PUGREST.ServerBusy'
Processing index 20
Error processing Ustekinumab: 'PUGREST.ServerBusy'
Processing index 21
Error processing Secukinumab: 'PUGREST.ServerBusy'
Processing index 22
Error processing Ixekizumab: 'PUGREST.ServerBusy'
Processing index 23
Error processing Apremilast: 'PUGREST.ServerBusy'
Processing index 24
Error processing Golimumab: 'PUGREST.ServerBusy'
Processing index 25
Error processing C